# BrowserUnit Experimentation Notebook

Currently in-development **BrowserUnit** is a simple python library class for running Selenium Webdriver tests inside UnitTest functions. It is intended to speed the automation of simple and repetitive browser actions.

This notebook is a collection of experimental methods to develop a minimal set of functions for quickly checking for element content, element presence, switching pages, parsing sitemaps, etc.


In [1]:
import rcf_lib as R
import unittest
import os
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options as C_Options
from selenium.webdriver.firefox.options import Options as FF_Options
import time
import sys
from colorama import init as cinit
cinit() # Filters ANSI escape sequences and inserts Win32 calls.

In [2]:
x = "Hello, world!"

In [3]:
print(x)

Hello, world!


In [57]:
# Assuming notebook was opened from py/
if( os.name == 'nt' ):
    print("Grabbing WinDOS executables...")
    FF_BINARY_PATH = "webdrivers/geckodriver.exe"
    CHROME_BINARY_PATH = "webdrivers/chromedriver.exe"
else:
    print("Grabbing Unix binaries...")
    FF_BINARY_PATH = "webdrivers/geckodriver"
    CHROME_BINARY_PATH = "webdrivers/chromedriver"
    
# In the future, throw errors if os.path to webdrivers folder/ drivers are not available.

def FF_DRIVER():
    firefox_options = FF_Options()
    return webdriver.Firefox(executable_path=FF_BINARY_PATH,options=firefox_options)

def C_DRIVER():
    chrome_options = C_Options()
    chrome_options.add_argument("--window-size=200,")
    chrome_options.add_argument("disable-infobars")
    chrome_options.add_argument("--disable-notifications")
    return webdriver.Chrome(executable_path=CHROME_BINARY_PATH,options=chrome_options)

# In final implementation, instantiate these drivers in test objects with bottlers (see #Implementation Plan)

Grabbing WinDOS executables...


At this point, `FF_DRIVER()` will open a FireFox window for testing, and `CHROME_DRIVER()` will open a Chrome window for testing. Both functions return the **driver** that is used for maniuplating the page. We'll attempt to open and manipulate the FireFox driver first. A window should open after this command is run.

In [5]:
# Test ChromeDriver
c_driver = C_DRIVER()
c_driver.get("http://catb.org/jargon/html/index.html")
print("Jargon" in c_driver.title) # Will print "True" if page loads successfully.
print(c_driver.title)
c_driver.close()

True
The Jargon File


In [6]:
# Test GeckoDriver
ff_driver = FF_DRIVER()
ff_driver.get("http://catb.org/jargon/html/index.html")
print("Jargon" in ff_driver.title) # Will print "True" if page loads successfully.
print(ff_driver.title)
ff_driver.close()

True
The Jargon File


### Requirements for the BrowserUnit Class:

1. Provide an abstraction layer for easy testing of website components.
2. Provide a system for defining the composition of a website's html so components can be isolated and verified as present or with certain properties.
3. Provide methods to check for elements and text on the page, in such a way that a script can be quickly constructed for testing against a requirement.
4. Provide a framework to write test cases for each requirement defined in a separate document of test cases.

### Stretch Functionality:

1. Pull and process sitemap using beautifulsoup , requests.

# Implementation Plan

In order to execute a series of pre-determined functions in order with variables, I will add the functions to a queue and execute them synchronously. I don't necessarily want to implement threading yet, but if I implement this correctly, it shouldn't be hard.

In [7]:
class Bottler:
    """Attempt to execute a stored array of functions."""
    def __init__(self):
        self.bottlingQueue = []

    def addTest(self, printme):
        def bottledTest():
            print( printme )
        self.bottlingQueue.append(bottledTest)

    def executeTests(self):
        for test in self.bottlingQueue:
            test()

b = Bottler()
b.addTest("One")
b.addTest("Two")
b.addTest("Three")
b.executeTests()

One
Two
Three


In [10]:
b.addTest("Four")
b.executeTests()

One
Two
Three
Four


`Bottler` demonstrates that functions can be stored as objects, stored in an array and called sequentially.

In [69]:
def initDriver( browser ):
    
    print(C_DRIVER)
    print(FF_DRIVER)
    
    if(str(browser).lower() =="chr"):
        return C_DRIVER
    elif(str(browser).lower() =="fir"):
        return FF_DRIVER

# Required Global Variables?
DRIVER = FF_DRIVER # initDriver("firefox") # Not sure why initDriver isn't functional.
DEBUG = True
UNITTESTS = []

# Eventually move to array of BrowserUnits
class UnitTestAtom(unittest.TestCase):
    
    def setUp(self):
        self.driver = DRIVER()
        self.a = "Lol"
        print("Setup")
    
    def tearDown(self):
        self.driver.close()
    
    def test_000_is_this_working(self):
        self.assertTrue( 1 > 0.9 )
    
    def ok(self):
        print("UnitTestAtom Instantiated.")

def BuildUnitTestAtom( name ):
    TestAtom = type(name, (UnitTestAtom,),{})
    return TestAtom() #Ensure the active class name is returned.

```
class BrowserUnitQueue(unittest.TestLoader):
    """Used for storing and running a collection of tests. One queue per Feature."""
    def __init__(self):
        self.test_queue = []
    
    def add_unit(self, x):
        self.test_queue.append(x)
    
    def list_units(self):
        print("")
        for x in self.test_queue:
            print(str(x))
```

In [54]:
class BrowserUnit():
    """Used for building a single unit-test."""
    def __init__(self, testname):
        self.testcase = BuildUnitTestAtom(str(testname))
        self.testcase.ok()
        self.execute()
        self.functionQueue = []
        UNITTESTS.append(self)
    
    def assertTitle(self, string):
        def bottledAssertTitle(string):
            self.assertIn(self.driver.title())
        
        self.functionQueue.append(bottledAssertTitle)
        
    def execute(self):
        """Do not use."""
        # unittest.TestSuite(tests=(self.testcase)).run()
        suite = unittest.TestLoader().loadTestsFromTestCase(self.testcase.__class__)
        unittest.TextTestRunner(verbosity=2).run(suite)

Basic tests for **BrowserUnit**:

In [70]:
test1 = BrowserUnit("test_one")

UnitTestAtom Instantiated.


test_000_is_this_working (__main__.test_one) ... 

Setup


ok

----------------------------------------------------------------------
Ran 1 test in 7.085s

OK


Now that we can store a unittest in BrowserUnit, let's see if we can run a few BrowserUnits with their internal TestAtoms:

In [47]:
UNITTESTS = [] # Clear previous unittests.

test1 = BrowserUnit("test_one")
test2 = BrowserUnit("test_two")
test3 = BrowserUnit("test_three")

for x in UNITTESTS:
    print(x)
    print(x.testcase)
    print(x.testcase.__class__.__name__)

UnitTestAtom Instantiated.
UnitTestAtom Instantiated.
UnitTestAtom Instantiated.
runTest (__main__.test_one)
test_one
runTest (__main__.test_two)
test_two
runTest (__main__.test_three)
test_three


In [28]:
Loader = unittest.TestLoader()



for x in UNITTESTS:
    Suite = Loader.loadTestsFromTestCase(x.testcase)
    unittest.TextTestRunner().run(Suite)
    # x.testcase.run('test_000_is_this_working')
    pass
    
# suite = unittest.defaultTestLoader.loadTestsFromTestCase(test1.testcase)
# unittest.TextTestRunner().run(suite)


TypeError: issubclass() arg 1 must be a class

_Fin_

Jupyter Notebook fidgeting:

In [ ]:
x = print
x("lol") # Oh, damn. Functions are objects.

# BROWSER_UNIT_QUEUE = BrowserUnitQueue()
# BROWSER_UNIT_QUEUE.add_unit("This ought to be a unittest obj.")
# BROWSER_UNIT_QUEUE.list_units()

In [ ]:
%pylab inline
x = linspace(0, 2*pi)
plot(x, sin(x), label=r'$\sin(x)$')
plot(x, cos(x), 'ro', label=r'$\cos(x)$')
title(r'Sine and Cosine')
legend() # Wow, this graph is pretty magical.